In [2]:
import pandas as pd

In [3]:
# ms = Messstellen
# ws = Wasserstände

cols_ms = ['messstelle_id', 'name', 'e32', 'n32', 'turnus_wasserstand', 'messstellenart',
           'freigabe_wstd', 'freigabe_chemie', 'freigabe_lage']
cols_ws = ['messstelle_id', 'datum_messung', 'abstich_m', 'hinweis', 'wasserstd_m', 'flurabstd_m', 'mph_m', 'gok_m', 'wwj']

cols_ms_dtype = {
    1: 'object', # messstelle_id
    19: 'category', # turnus_wasserstand
    20: 'category', # freigabe_wstd
    21: 'category', # freigabe_chemie
    22: 'category', # freigabe_lage
    27: 'category' # messstellenart
}

cols_ws_dtype = {
    1: 'object', # messstelle_id
    2: 'int32', # messstelle_sl_nr
    4: 'float32', # abstich_m
    5: 'object', # hinweis
    6: 'float32', # wasserstd_m
    7: 'float32', # flurabstd_m
    8: 'float32', # mph_m
    9: 'float32', #gok_m
    10: 'int32' # wwj
    }


In [4]:
df_ms = pd.read_csv("../data/raw/grundwasserdaten-NRW/OpenHygrisC_gw-messstelle.csv", 
                    encoding="utf-8", sep=";", decimal=",", thousands='.', usecols=cols_ms, dtype=cols_ms_dtype)
df_ws = pd.read_csv("../data/raw/grundwasserdaten-NRW/OpenHygrisC_gw-wasserstand_2010-2019.csv", 
                    encoding="utf-8", parse_dates=['datum_messung'], sep=";", decimal=',', thousands='.',
                    usecols=cols_ws, dtype = cols_ws_dtype, na_values=[""])

In [5]:
df_ms.dtypes


messstelle_id           object
name                    object
e32                     object
n32                     object
turnus_wasserstand    category
freigabe_wstd         category
freigabe_chemie       category
freigabe_lage         category
messstellenart        category
dtype: object

In [6]:
df_ms.head(10)

,messstelle_id,name,e32,n32,turnus_wasserstand,freigabe_wstd,freigabe_chemie,freigabe_lage,messstellenart
0,086610363,STRAM3,3071XX,56990XX,Einzelmessung,ja,nein,nein,GW-Messstelle
1,030302419,Monh-Baumberg,353497,5665195,Messstelle inaktiv,ja,ja,ja,GW-Messstelle
2,070283916,LGD BN KOTTENF. 3,362354,5615873,monatlich,ja,ja,ja,GW-Messstelle
3,010445730,60BP010505,2911XX,56692XX,vierzehntäglich,ja,nein,nein,GW-Messstelle
4,010445791,60BP010701,2908XX,56657XX,vierzehntäglich,ja,nein,nein,GW-Messstelle
5,076981812,RSK Wahner H. WH18,3688XX,56333XX,Messstelle inaktiv,ja,nein,nein,GW-Messstelle
6,010446722,60GP012301,2920XX,56453XX,monatlich,ja,nein,nein,GW-Messstelle
7,100151115,WE 33 WEHE,4787XX,58085XX,monatlich,ja,nein,nein,GW-Messstelle
8,073917310,K WA 37,3646XX,56364XX,halbjährlich,ja,nein,nein,GW-Messstelle
9,100604511,"849 MLK 102,8S",4956XX,57947XX,Messstelle inaktiv,ja,nein,nein,GW-Messstelle


In [20]:
print('NaN-Werte Messstellen')
display(df_ms.isna().sum())
print('---------------------')
print('NaN-Werte Wasserstände')
display(df_ws.isna().sum())

NaN-Werte Messstellen


messstelle_id         0
name                  0
e32                   0
n32                   0
turnus_wasserstand    0
freigabe_wstd         0
freigabe_chemie       0
freigabe_lage         0
messstellenart        0
dtype: int64

---------------------
NaN-Werte Wasserstände


messstelle_id        0
datum_messung        0
abstich_m            0
hinweis              0
wasserstd_m          0
flurabstd_m      33427
mph_m                0
gok_m            33424
wwj                  0
dtype: int64

In [8]:
# Wenn der Turnus der Wasserstandsmessung NaN ist,
# findet die Messstelle keine Beachtung
df_ms = df_ms.dropna(subset=['turnus_wasserstand'])

# Wenn der Wasserstand NaN-Werte hat, ist die GW-Messstelle trocken.
# Diese GW-Messstellen sollen nicht näher betrachtet werden.
df_ws = df_ws.dropna(subset=['wasserstd_m'])

In [22]:
print('NaN-Werte Messstellen')
display(df_ms.isna().sum())
print('---------------------')
print('NaN-Werte Wasserstände')
display(df_ws.isna().sum())

NaN-Werte Messstellen


messstelle_id         0
name                  0
e32                   0
n32                   0
turnus_wasserstand    0
freigabe_wstd         0
freigabe_chemie       0
freigabe_lage         0
messstellenart        0
dtype: int64

---------------------
NaN-Werte Wasserstände


messstelle_id        0
datum_messung        0
abstich_m            0
hinweis              0
wasserstd_m          0
flurabstd_m      33427
mph_m                0
gok_m            33424
wwj                  0
dtype: int64

In [10]:
# Correction of missing coordinates. Replace XX with the mean value of 50. For plotting in GIS.
mask = df_ms['freigabe_lage'] == 'nein'

df_ms.loc[mask, 'e32'] = df_ms.loc[mask, 'e32'].str.replace("XX", "50")
df_ms.loc[mask, 'n32'] = df_ms.loc[mask, 'n32'].str.replace("XX", "50")

In [11]:
df_ms.head(10)

,messstelle_id,name,e32,n32,turnus_wasserstand,freigabe_wstd,freigabe_chemie,freigabe_lage,messstellenart
0,086610363,STRAM3,307150,5699050,Einzelmessung,ja,nein,nein,GW-Messstelle
1,030302419,Monh-Baumberg,353497,5665195,Messstelle inaktiv,ja,ja,ja,GW-Messstelle
2,070283916,LGD BN KOTTENF. 3,362354,5615873,monatlich,ja,ja,ja,GW-Messstelle
3,010445730,60BP010505,291150,5669250,vierzehntäglich,ja,nein,nein,GW-Messstelle
4,010445791,60BP010701,290850,5665750,vierzehntäglich,ja,nein,nein,GW-Messstelle
5,076981812,RSK Wahner H. WH18,368850,5633350,Messstelle inaktiv,ja,nein,nein,GW-Messstelle
6,010446722,60GP012301,292050,5645350,monatlich,ja,nein,nein,GW-Messstelle
7,100151115,WE 33 WEHE,478750,5808550,monatlich,ja,nein,nein,GW-Messstelle
8,073917310,K WA 37,364650,5636450,halbjährlich,ja,nein,nein,GW-Messstelle
9,100604511,"849 MLK 102,8S",495650,5794750,Messstelle inaktiv,ja,nein,nein,GW-Messstelle


In [12]:
df_ws.head(10)

,messstelle_id,datum_messung,abstich_m,hinweis,wasserstd_m,flurabstd_m,mph_m,gok_m,wwj
1,218086015,2019-10-13,0.510000,-,74.760002,-0.440000,75.269997,74.320000,2019
2,010305970,2019-08-12,3.440000,-,41.070000,2.710000,44.509998,43.779999,2019
3,219610927,2012-05-22,22.840000,-,75.440002,22.950001,98.279999,98.389999,2012
4,219610927,2012-06-27,22.780001,-,75.500000,22.889999,98.279999,98.389999,2012
5,219610927,2012-07-26,22.799999,-,75.480003,22.910000,98.279999,98.389999,2012
6,219610927,2012-08-21,22.830000,-,75.449997,22.940001,98.279999,98.389999,2012
7,219610927,2012-09-25,22.799999,-,75.480003,22.910000,98.279999,98.389999,2012
8,219610927,2012-10-24,22.809999,-,75.470001,22.920000,98.279999,98.389999,2012
9,219610927,2012-11-21,22.840000,-,75.440002,22.950001,98.279999,98.389999,2013
10,219610927,2013-01-03,22.850000,-,75.430000,22.959999,98.279999,98.389999,2013


In [13]:
df_ms_cross = pd.crosstab(index=df_ms.loc[:, 'messstellenart'], columns=df_ms.loc[:, 'turnus_wasserstand'])

df_ms_cross

turnus_wasserstand,-,2-monatlich,Einzelmessung,Messstelle besteht nicht mehr,Messstelle inaktiv,halbjährlich,jährlich,monatlich,täglich,vierteljährlich,vierzehntäglich,vierzehntäglich (gerade Wochen),vierzehntäglich (ung. Wochen),wöchentlich
messstellenart,,,,,,,,,,,,,,
Bergbauschacht,0,0,0,0,0,7,0,25,11,7,0,0,0,0
Bodenaufschlussbohrungen,0,0,2,686,2,0,0,0,2,0,0,0,0,0
GW-Messstelle,2160,431,2402,7431,18204,3660,464,22497,1692,714,864,180,61,990
Grundwasserblänke,0,0,0,1,59,6,0,19,1,1,0,0,0,3
Horizontalfilterbrunnen,0,0,8,1,28,6,11,11,1,0,1,0,0,4
Mehrfachmessstelle,0,0,2,1,0,2,2,12,1,0,0,0,0,1
Pegel,0,9,13,45,174,19,0,336,21,14,7,0,0,51
Rohmischwassermessstelle,0,0,0,1,40,0,0,1,0,0,0,0,0,0
Sammelmessstelle,0,0,0,0,34,1,0,1,0,0,0,0,0,0


In [14]:
# Boolean Mask with all conditions
df_ms_mask = (
    (df_ms.loc[:, 'messstellenart'].str.strip() == 'GW-Messstelle') & 
    ((df_ms['turnus_wasserstand'] == 'monatlich') | (df_ms['turnus_wasserstand'] == 'täglich') |
     (df_ms['turnus_wasserstand'] == 'vierzehntäglich') | (df_ms['turnus_wasserstand'] == 'wöchentlich')) & 
    (df_ms.loc[:, 'freigabe_wstd'] == 'ja')
) 

In [15]:
df_ms_ws_m = pd.merge(
    df_ms[df_ms_mask],
    df_ws,
    on="messstelle_id",
    how="inner"
)

df_ms_ws_m.head(5)

,messstelle_id,name,e32,n32,turnus_wasserstand,freigabe_wstd,freigabe_chemie,freigabe_lage,messstellenart,datum_messung,abstich_m,hinweis,wasserstd_m,flurabstd_m,mph_m,gok_m,wwj
0,070283916,LGD BN KOTTENF. 3,362354,5615873,monatlich,ja,ja,ja,GW-Messstelle,2016-03-21,6.17,-,158.779999,5.56,164.949997,164.339996,2016
1,070283916,LGD BN KOTTENF. 3,362354,5615873,monatlich,ja,ja,ja,GW-Messstelle,2016-04-21,5.95,-,159.000000,5.34,164.949997,164.339996,2016
2,070283916,LGD BN KOTTENF. 3,362354,5615873,monatlich,ja,ja,ja,GW-Messstelle,2016-05-24,5.88,-,159.070007,5.27,164.949997,164.339996,2016
3,070283916,LGD BN KOTTENF. 3,362354,5615873,monatlich,ja,ja,ja,GW-Messstelle,2016-06-24,5.82,-,159.130005,5.21,164.949997,164.339996,2016
4,070283916,LGD BN KOTTENF. 3,362354,5615873,monatlich,ja,ja,ja,GW-Messstelle,2016-07-25,5.84,-,159.110001,5.23,164.949997,164.339996,2016


In [16]:
print(f"Anzahl der relevanten Messstellen in der messstellen.csv: {len(df_ms[df_ms_mask].loc[:, 'messstelle_id'].unique())}")
print(f'Anzahl aller Wasserstände in der wasserstand.csv: {len(df_ws.loc[:, 'messstelle_id'].unique())}')
print('---------------------------------------------------------')
print(f'Anzahl der resultierenden Messstellen aus merge: {len(df_ms_ws_m.loc[:, 'messstelle_id'].unique())}')

Anzahl der relevanten Messstellen in der messstellen.csv: 12384
Anzahl aller Wasserstände in der wasserstand.csv: 18494
---------------------------------------------------------
Anzahl der resultierenden Messstellen aus merge: 11348


In [17]:
# Selection of measuring points where sufficient groundwater levels are present
df_selected_ms = (
    df_ms_ws_m
    .drop_duplicates(subset="messstelle_id")
    .drop(columns=["datum_messung", "abstich_m", "hinweis", "wasserstd_m", "flurabstd_m", "mph_m", "gok_m", "wwj"])
    .reset_index(drop=True)
    )

In [18]:
df_ms_ws_m.dtypes

messstelle_id                 object
name                          object
e32                           object
n32                           object
turnus_wasserstand          category
freigabe_wstd               category
freigabe_chemie             category
freigabe_lage               category
messstellenart              category
datum_messung         datetime64[ns]
abstich_m                    float32
hinweis                       object
wasserstd_m                  float32
flurabstd_m                  float32
mph_m                        float32
gok_m                        float32
wwj                            int32
dtype: object

In [19]:
# Export DataFrame to CSV and Pickle
df_ms.to_pickle("../data/processed/grundwasserdaten-NRW/GW-Messstellen.pkl")
df_ms.to_csv("../data/processed/grundwasserdaten-NRW/GW-Messstellen.csv")
df_ws.to_pickle("../data/processed/grundwasserdaten-NRW/GW-Wasserstaende.pkl")
df_ms_ws_m.to_pickle("../data/processed/grundwasserdaten-NRW/GW-MessWasserstellenMerge.pkl")
df_selected_ms.to_csv("../data/processed/grundwasserdaten-NRW/GW-MessstellenSelected.csv")